<a href="https://colab.research.google.com/github/saurabhthesuperhero/NLP/blob/main/ChatBot_Deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [37]:
with open("intents.json") as file:
  data=json.load(file)
training_sentences=[]
training_labels=[]
labels=[]
responses=[]

for intent in data['intents']:
  for pattern in intent['patterns']:
    training_sentences.append(pattern)
    training_labels.append(intent['tag'])
  responses.append(intent['responses'])


  if(intent['tag'] not in labels):
    labels.append(intent['tag'])

num_classes=len(labels)


In [38]:
print(training_sentences)
print(training_labels)
print(labels)
print(responses)

['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay', 'Bye', 'See you later', 'Goodbye', 'Thanks', 'Thank you', "That's helpful", 'Thanks for the help', 'Who are you?', 'What are you?', 'Who you are?', 'what is your name', 'what should I call you', 'whats your name?', 'Could you help me?', 'give me a hand please', 'Can you help?', 'What can you do for me?', 'I need a support', 'I need a help', 'support me please', 'I need to create a new account', 'how to open a new account', 'I want to create an account', 'can you create an account for me', 'how to open a new account', 'have a complaint', 'I want to raise a complaint', 'there is a complaint about a service']
['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'goodbye', 'goodbye', 'goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'about', 'about', 'about', 'name', 'name', 'name', 'help', 'help', 'help', 'help', 'help', 'help', 'help', 'createaccount', 'createaccount', 'createaccount', 'createaccount', 'createaccount', 'complaint

In [42]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels2 = lbl_encoder.transform(training_labels)
print(training_labels2)  

[4 4 4 4 4 3 3 3 7 7 7 7 0 0 0 6 6 6 5 5 5 5 5 5 5 2 2 2 2 2 1 1 1]


In [40]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d_3 ( (None, 16)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_11 (Dense)             (None, 8)                 136       
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
_________________________________________________________________


In [43]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels2), epochs=epochs)

Epoch 1/500
2/2 [==============================] - 0s 44ms/step - loss: 2.0821 - accuracy: 0.1212
Epoch 2/500
2/2 [==============================] - 0s 27ms/step - loss: 2.0794 - accuracy: 0.1515
Epoch 3/500
2/2 [==============================] - 0s 39ms/step - loss: 2.0778 - accuracy: 0.1515
Epoch 4/500
2/2 [==============================] - 0s 34ms/step - loss: 2.0764 - accuracy: 0.1515
Epoch 5/500
2/2 [==============================] - 0s 31ms/step - loss: 2.0756 - accuracy: 0.1515
Epoch 6/500
2/2 [==============================] - 0s 25ms/step - loss: 2.0751 - accuracy: 0.1515
Epoch 7/500
2/2 [==============================] - 0s 24ms/step - loss: 2.0746 - accuracy: 0.1515
Epoch 8/500
2/2 [==============================] - 0s 43ms/step - loss: 2.0740 - accuracy: 0.1515
Epoch 9/500
2/2 [==============================] - 0s 23ms/step - loss: 2.0732 - accuracy: 0.1515
Epoch 10/500
2/2 [==============================] - 0s 39ms/step - loss: 2.0724 - accuracy: 0.1515
Epoch 11/500
2/2 [=

In [44]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets


In [ ]:
t

In [46]:

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        inp = input()
        if inp.lower() == "quit":
            break

        result = model(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])
        # print(tag)
        for i in data['intents']:
          # print(222)
          # print(i['tag'],tag,[np.argmax(result)])
          if i['tag'] == tag:
            # print(22)
            print( "ChatBot:", np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
hiiiiii
ChatBot: Hi
hello
ChatBot: Hi there
what are you
ChatBot: I.m Joana, your bot assistant
what you can do
ChatBot: Tell me how can assist you
tell me song
ChatBot: Tell me your problem to assist you
hey chat
ChatBot: Hi
your name
ChatBot: Just call me as Joana
quit


In [15]:
pip install colorama